## More about functions

We have presented the foundations on which the language is based in terms of functions:

- Functions as first-class citizens in the language
- Composition of functions
- Partial application

Let's see some more examples to progress

In [1]:
let fullName firstName lastName =
    let s = firstName + lastName 
    s 

let john = fullName "John" "Lennon"     

printfn "%A" john    


"JohnLennon"


We see that other identifiers can be defined within functions, just as we expected. In this case, apparently, the function would concatenate the strings `firstName` and `lastName` into a single string, which it returns as its result via `s`.

However, an interesting point to note with the `fullName` function is that until the function is applied to two strings, the compiler has no way of knowing what data type I want to use. This is because the `+` operator in F# is used to both add numbers and concatenate _strings_. Although the name of the function suggests that _strings_ will be used, the compiler does not do a semantic analysis...


In [2]:
let fullNameS firstName lastName =
    let s = firstName + " " +  lastName 
    s 

In the case of `fullNameS` the compiler correctly infers the types as _strings_, since clearly the `+` operator is being used for concatenation, when used with ` + " " +`.

> Here we must clarify a peculiarity of _Notebooks_: if I define the function in one cell, and use it in another, the function will be associated to inputs (and outputs) of integer type. In order for the compiler to correctly infer the data types, I need to use the function on the same cell. From there, the data types will be defined for the entire _Notebook_.

Of course you can call functions inside other functions:

In [3]:
let prettyPrint s =
    printfn "%A" s 

let hola name =
    prettyPrint ("Hola " + name + " !")

In [4]:
let paul = fullNameS "Paul" "McCartney"

prettyPrint paul

"Paul McCartney"


In [5]:
hola paul

"Hola Paul McCartney !"


In [6]:
paul
|> hola

"Hola Paul McCartney !"


We use parentheses to separate function calls from function calls:

In [6]:
hola (fullNameS "George" "Harrison")

"Hola George Harrison !"


We can also use the _pipe_ operator:

In [7]:
paul 
|> hola 

"Hola Paul McCartney !"


But be careful when you use several inputs, because _piping_ only accepts the last argument when there is more than one:

In [7]:
"George" "Harrison"
|> fullNameS 
|> hola 

Error: input.fsx (1,1)-(1,9) typecheck error This value is not a function and cannot be applied.
input.fsx (3,4)-(3,8) typecheck error Type mismatch. Expecting a
    '(string -> string) -> obj'    
but given a
    'string -> unit'    
The type 'string -> string' does not match the type 'string'

In [9]:
"Harrison"
|> fullNameS "George" 
|> hola 

"Hola George Harrison !"


Let's see a slightly clearer example. Returning to our vending machine, suppose the cost of a package of ACME brand cookies is known, which is the price paid by the vendor. A percentage of profit is applied to this cost, and also another percentage of taxes that is transferred to the final consumer. Let's see how to calculate the package price:

In [10]:
let priceAfterTax tax value = 
    let price = value * (1.0 + tax/100.0)
    price 

let priceAfterEarning earning value = 
    let price = value * (1.0 + earning/100.0)
    price 

let price tax earning value = 
    let p = priceAfterTax tax value 
    let q = priceAfterEarning earning p 
    q  



In [12]:
let cost = 45.0 

let tax = 21 
let earning = 10 

let finalPrice = price tax earning cost 

prettyPrint finalPrice

59.895


Or one could use pipe:

In [13]:
cost
|> priceAfterEarning earning 
|> priceAfterTax tax 


59.89500000000001

Another possibility is to generalize and create a single function to calculate the percentage

In [15]:
let applyPercentage per value = 
    value * (1.0 + per/100.0)

let priceAfterEarning2 = applyPercentage earning
let priceAfterTax2 = applyPercentage tax     

In [16]:
cost
|> priceAfterEarning2
|> priceAfterTax2 

59.89500000000001

Which is equivalent to composing the functions:

In [17]:
let finalPrice = priceAfterEarning2 >> priceAfterTax2 

In [18]:
printfn "%A" (finalPrice cost)

59.895
